---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
#import matplotlib.pyplot as plt
def graph_identification():
    
    # Your Code Here
    for j in range(0,5):
        G = P1_Graphs[j]
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
        
        print (j,'\t',nx.density(G),'\t',nx.diameter(G),'\t',nx.average_clustering(G),'\t',nx.average_shortest_path_length(G))
        '''
        plt.plot(degree_values,histogram, 'o')
        plt.xlabel('Degree')
        plt.ylabel('Fraction of Nodes')
        plt.xscale('log')
        plt.yscale('log')
        plt.show()
        '''
    return ['PA', 'SW_H', 'SW_L','PA', 'PA'] # Your Answer Here
graph_identification()

0 	 0.003995995995995996 	 7 	 0.03167539146454044 	 4.099161161161161
1 	 0.01001001001001001 	 9 	 0.5642419635919628 	 5.089871871871872
2 	 0.0053404539385847796 	 20 	 0.4018222222222227 	 9.378702269692925
3 	 0.010623943035157988 	 5 	 0.03780379975223251 	 3.1048046283934134
4 	 0.0053404539385847796 	 9 	 0.0033037037037037037 	 5.0785509568313305


['PA', 'SW_L', 'SW_H', 'PA', 'PA']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [ ]:
G = nx.read_gpickle('email_prediction.txt')

#print(nx.info(G))

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
def salary_predictions():
    
    # Your Code Here
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(nx.degree_centrality(G))
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G,normalized = True, endpoints = False))
    df['rank'] = pd.Series(nx.pagerank(G, alpha=0.85))
    hits = nx.hits(G)
    df['hub'] = pd.Series(hits[0])
    df['authority'] = pd.Series(hits[1])
    df_res = df[df['ManagementSalary'].isnull() != 0]
    df.dropna(inplace=True)
    X = df[['clustering','degree','betweenness','rank','hub','authority']]
    y = df[['ManagementSalary']]
    #sklearn
    seed = 42
    X_train,X_test,y_train,y_test = train_test_split( X , y , train_size = .8, random_state = seed)
    model = GradientBoostingClassifier(random_state = seed)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    X_res = df_res[['clustering','degree','betweenness','rank','hub','authority']]
    y_res = df_res[['ManagementSalary']]
    y_res = model.predict_proba(X_res)
    res_list = []
    for i in range(0,y_res.shape[0]):
        res_list.append(y_res[i][1])
    res = pd.Series(res_list,index = df_res.index )
    return res# Your Answer Here

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
#future_connections.head(10)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
;        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    
    # Your Code Here
    bb = nx.get_node_attributes(G, 'Department')
    nx.set_node_attributes(G,'community',bb)
    
    #Building features
    future_connections['Jaccard_coef'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['res_alloc'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    future_connections['pref_attach'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['community_neigh'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G,future_connections.index)]
    future_connections['community_resource'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G,future_connections.index)]
    
    #Separating empty values
    fc_res = future_connections[future_connections['Future Connection'].isnull() != 0]
    future_connections.dropna(inplace=True)
    
    #X,y
    X = future_connections[['Jaccard_coef','res_alloc','pref_attach','community_neigh','community_resource']]
    y = future_connections[['Future Connection']]
    
    #sklearn
    seed = 42
    X_train_un,X_test_un,y_train,y_test = train_test_split( X , y , train_size = .8, random_state = seed)
    
    #Scaling values
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train_un)
    X_test = scaler.transform(X_test_un)
    
    #Machine learning model
    model = GradientBoostingClassifier(random_state = seed)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    #Predicting values for result
    X_res_un = fc_res[['Jaccard_coef','res_alloc','pref_attach','community_neigh','community_resource']]
    y_res = fc_res[['Future Connection']]
    X_res = scaler.transform(X_test_un)
    y_res = model.predict_proba(X_res_un)
    
    #Proper format for submission
    res_list = []
    for i in range(0,y_res.shape[0]):
        res_list.append(y_res[i][1])
    res = pd.Series(res_list,index = fc_res.index )
    return res# Your Answer Here